In [24]:
import os
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dense, Flatten
import tensorflow.python.keras.optimizers
import tensorflow.python.keras.metrics

import keras_tuner as kt

#%%capture
data_file_path = os.path.join(os.getcwd(), "Data\\returnsData.csv")
returnsData = pd.read_csv(data_file_path)
#returnsData.info(verbose=True)
longReturns = returnsData[["YearQuarter", "gvkey", "quarterlyReturns"]] #"quarterlyVolatility",
#longReturns.info(verbose=True)
wideReturns = pd.pivot(longReturns, index="YearQuarter", columns="gvkey", values="quarterlyReturns")
#wideReturns.info(verbose=True)
#print(wideReturns.iloc[0:10,0:10])

sample_length = 10
dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=wideReturns, targets=None,
    sequence_length=sample_length, sequence_stride=1, sampling_rate=1,
    batch_size=128,  shuffle=False)

for batch in dataset:
  data = batch
  #assert np.array_equal(inputs[0], wideReturns[:sample_length])
  # second sample equals output timestamps 20-40
  #assert np.array_equal(targets[1], wideReturns[sample_length:2*sample_length])
  break

# targets = data[:,-1,:]
# inputs = data[:,:-1,:]

# a = inputs[..., 0:20]
#print(inputs[..., 0:10])
#print(targets.iloc[0:10,0:10])

# #[lags of YearQuarter, YearQuarter, gvkey] => [Batch, YearQuarter, None]
# finalData = np.transpose(a=inputs, axes = (2,0,1))
# finalData = np.reshape(finalData, (-1, sample_length-1))
# #b = finalData[0:50,:]



finalData = np.transpose(a=data, axes = (2,0,1))
finalData = np.reshape(finalData, (-1, sample_length))
# input = finalData[:,:-1]
# targets = finalData[:,-1]



xTrain, xVal, xTest = finalData[:,:-1], 0, 0
yTrain, yVal, yTest = finalData[:,-1], 0, 0

model = Sequential()
model.add(tf.keras.layers.LSTM(units=5, input_shape=(sample_length-1,)))    #mask
model.add(Dense(units=1))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss="MeanAbsoluteError",
              metrics=[tf.keras.losses.MeanSquaredError(), tf.keras.losses.MeanAbsoluteError()])

model.fit(x=xTrain, y=yTrain, batch_size=32)

# class MyHyperModel(kt.HyperModel):
#     def build(self, hp):
#         model = Sequential()
#         model.add(tf.keras.layers.LSTM(units=hp.Int("units", min_value=32, max_value=512, step=32), input_shape=(sample_length-1,)))
#         model.add(Dense(units=1))

#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=hp.Int("learning_rate", [16, 32])),
#                   loss="MeanAbsoluteError",
#                   metrics=[tf.keras.losses.MeanSquaredError(), tf.keras.losses.MeanAbsoluteError()])
#         return model

#     def fit(self, hp, model, *args, **kwargs):
#         return model.fit(
#             *args,
#             batch_size=hp.Choice("batch_size", [16, 32]),
#             **kwargs,
#         )

# tuner = kt.RandomSearch(
#     MyHyperModel(),
#     objective="val_accuracy",
#     max_trials=3,
#     overwrite=True,
#     directory="my_dir",
#     project_name="tune_hypermodel",
# )




ValueError: Could not interpret optimizer identifier: <keras.optimizers.optimizer_v2.adam.Adam object at 0x000001142660A0B0>